<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Objective

The goal of this notebook is to create appropriate files and place them in azure in preparation for running the parallel scoring AML pipeline. This is the second of three notebooks.

The order of the execution of these notebooks should be:

- [parallel_scoring_0_prepare_azure_resources](parallel_scoring_0_prepare_azure_resources.ipynb): This creates the azure resources used in other notebooks. This should already be run when this notebook is executed.
- [parallel_scoring_1_prepare_data_and_model](parallel_scoring_1_prepare_data_and_model.ipynb) **(This notebook)**: This uploads ratings data and an example model to Azure where agents can see them.
- [parallel_scoring_2_prepare_and_run_amlpipeline](parallel_scoring_2_prepare_and_run_amlpipeline.ipynb): This creates and executes a parallel pipeline that leverages the resources and files created in prior steps.

# SAR Single Node on MovieLens (Python, CPU)

In this example, we will walk through each step of the Smart Adaptive Recommendations (SAR) algorithm using a Python single-node implementation.

SAR is a fast, scalable, adaptive algorithm for personalized recommendations based on user transaction history. It is powered by understanding the similarity between items, and recommending similar items to those a user has an existing affinity for.

## 1 SAR algorithm

The following figure presents a high-level architecture of SAR. 

At a very high level, two intermediate matrices are created and used to generate a set of recommendation scores:

- An item similarity matrix $S$ estimates item-item relationships.
- An affinity matrix $A$ estimates user-item relationships.

Recommendation scores are then created by computing the matrix multiplication $A\times S$.

Optional steps (e.g. "time decay" and "remove seen items") are described in the details below.

<img src="https://recodatasets.blob.core.windows.net/images/sar_schema.svg?sanitize=true">

### 1.1 Compute item co-occurrence and item similarity

SAR defines similarity based on item-to-item co-occurrence data. Co-occurrence is defined as the number of times two items appear together for a given user. We can represent the co-occurrence of all items as a $m\times m$ matrix $C$, where $c_{i,j}$ is the number of times item $i$ occurred with item $j$, and $m$ is the total number of items.

The co-occurence matric $C$ has the following properties:

- It is symmetric, so $c_{i,j} = c_{j,i}$
- It is nonnegative: $c_{i,j} \geq 0$
- The occurrences are at least as large as the co-occurrences. I.e., the largest element for each row (and column) is on the main diagonal: $\forall(i,j) C_{i,i},C_{j,j} \geq C_{i,j}$.

Once we have a co-occurrence matrix, an item similarity matrix $S$ can be obtained by rescaling the co-occurrences according to a given metric. Options for the metric include `Jaccard`, `lift`, and `counts` (meaning no rescaling).


If $c_{ii}$ and $c_{jj}$ are the $i$th and $j$th diagonal elements of $C$, the rescaling options are:

- `Jaccard`: $s_{ij}=\frac{c_{ij}}{(c_{ii}+c_{jj}-c_{ij})}$
- `lift`: $s_{ij}=\frac{c_{ij}}{(c_{ii} \times c_{jj})}$
- `counts`: $s_{ij}=c_{ij}$

In general, using `counts` as a similarity metric favours predictability, meaning that the most popular items will be recommended most of the time. `lift` by contrast favours discoverability/serendipity: an item that is less popular overall but highly favoured by a small subset of users is more likely to be recommended. `Jaccard` is a compromise between the two.


### 1.2 Compute user affinity scores

The affinity matrix in SAR captures the strength of the relationship between each individual user and the items that user has already interacted with. SAR incorporates two factors that can impact users' affinities: 

- It can consider information about the **type** of user-item interaction through differential weighting of different events (e.g. it may weigh events in which a user rated a particular item more heavily than events in which a user viewed the item).
- It can consider information about **when** a user-item event occurred (e.g. it may discount the value of events that take place in the distant past.

Formalizing these factors produces us an expression for user-item affinity:

$$a_{ij}=\sum_k w_k e^{[-\text{log}(2)\frac{t_0-t_k}{T}]} $$

where the affinity $a_{ij}$ for user $i$ and item $j$ is the weighted sum of all $k$ events involving user $i$ and item $j$. $w_k$ represents the weight of a particular event, and the exponential term reflects the temporally-discounted event. The $\text{log}(2)$ scaling factor causes the parameter $T$ to serve as a half-life: events $T$ units before $t_0$ will be given half the weight as those taking place at $t_0$.

Repeating this computation for all $n$ users and $m$ items results in an $n\times m$ matrix $A$. Simplifications of the above expression can be obtained by setting all the weights equal to 1 (effectively ignoring event types), or by setting the half-life parameter $T$ to infinity (ignoring transaction times).

### 1.3 Remove seen items

Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

### 1.4 Top-k item calculation

The personalized recommendations for a set of users can then be obtained by multiplying the affinity matrix ($A$) by the similarity matrix ($S$). The result is a recommendation score matrix, where each row corresponds to a user, each column corresponds to an item, and each entry corresponds to a user / item pair. Higher scores correspond to more strongly recommended items.

It is worth noting that the complexity of the recommending operation depends on the data size. The SAR algorithm itself has $O(n^3)$ complexity. Therefore, the single-node implementation is not supposed to handle large datasets in a scalable manner. Whenever one uses the algorithm, it is recommended to run with sufficiently large memory. 

## 2 SAR single-node implementation

The SAR implementation illustrated in this notebook was developed in Python, primarily with Python packages like `numpy`, `pandas`, and `scipy` which are commonly used in most of the data analytics / machine learning tasks. Details of the implementation can be found in [Recommenders/reco_utils/recommender/sar/sar_singlenode.py](../../reco_utils/recommender/sar/sar_singlenode.py).

## 3 SAR single-node based movie recommender

## Load modules

In [ ]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import itertools
import logging
import os
import json
import pickle

## for uploading to azure
from azure.storage.blob import BlockBlobService

import numpy as np
import pandas as pd
import papermill as pm

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

In [ ]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '10m'

# Data path to where to write out parquet to
DATA_PATH = '../../data'

### 3.1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>, <Time>`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. 

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [ ]:
%%time
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=['UserId','MovieId','Rating','Timestamp']
)

# Convert the float precision to 32-bit in order to reduce memory consumption 
data.loc[:, 'Rating'] = data['Rating'].astype(np.float32)

data.head()

In [ ]:
data.dtypes

### 3.1.1 Persist Data for use by Azure Agents

Once we've created `data`, we will write it out as a parquet file, and then upload to blob storage, so that the scoring agents can access it without downloading it again.

In [ ]:
## read in data about azure resources:
amlpipeline_config = 'pipeline_config_programmatic.json'
with open(amlpipeline_config) as f:
    j = json.loads(f.read())

In [ ]:
## make sure data dir exists:
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

## serialize data to parquet:
parquet_name = 'ratings_%s.parquet.zip' %(MOVIELENS_DATA_SIZE)
full_path_parquet = os.path.join(DATA_PATH,parquet_name)
print('Writing to %s' %(full_path_parquet))
data.to_parquet(full_path_parquet, compression='gzip')

In [ ]:
## upload the data to Azure 

block_blob_service = BlockBlobService(account_name=j['blob_account'], account_key=j['blob_key']) 
block_blob_service.create_blob_from_path(container_name=j['data_blob_container'],
                                         blob_name=os.path.basename(full_path_parquet), 
                                         file_path=full_path_parquet)


In [ ]:
header = {
        "col_user": "UserId",
        "col_item": "MovieId",
        "col_rating": "Rating",
        "col_timestamp": "Timestamp",
    }

In this case, for the illustration purpose, the following parameter values are used:

|Parameter|Value|Description|
|---------|---------|-------------|
|`similarity_type`|`jaccard`|Method used to calculate item similarity.|
|`time_decay_coefficient`|30|Period in days (term of $T$ shown in the formula of Section 2.2.2)|
|`time_now`|`None`|Time decay reference.|
|`timedecay_formula`|`True`|Whether time decay formula is used.|

In [ ]:
# set log level to INFO
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SARSingleNode(
    remove_seen=True, 
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

## Compute inter-item similarity

Now we are ready to compute item-item similarity.

**NOTE**: This example is not intended to show how to estimate a model that would be used evaluate performance. In order to do that, we would split data into training and testing. This particular estimation is simply intended to provide an example of how to compute an intermediate model that can be leveraged in another environment. In other words, the estimated model below represents a situation where you already have a validated model, and you want to understand how to score it against potentially updated data in a routine manner.

In [ ]:
%%time 
model.fit0(data)

In [ ]:
## serialize the data
model_filename = 'sar_model_%s_fit0.pkl' %(MOVIELENS_DATA_SIZE)
full_model_path = os.path.join(DATA_PATH, model_filename)
print('writing model to %s' %(full_model_path))
with open(full_model_path,'wb') as fh:
    pickle.dump(model, fh, protocol = 4)

In [ ]:
## upload the data to Azure 
## disable logging for this:
logging.disable(logging.DEBUG);
logging.disable(logging.INFO);

block_blob_service = BlockBlobService(account_name=j['blob_account'], account_key=j['blob_key']) 
block_blob_service.create_blob_from_path(container_name=j['models_blob_container'],
                                         blob_name=os.path.basename(full_model_path), 
                                         file_path=full_model_path)


## Continue...

At this point, you have collected your data, you have estimated part of your model, you have uploaded them to an azure storage account, and you are ready to do the next piece, which is to leverage Azure Machine Learning to score chunks of the data. To see that, please continue to the parallel scoring notebook [here](parallel_scoring_2_prepare_and_run_amlpipeline.ipynb).

## References
Note SAR is a combinational algorithm that implements different industry heuristics. The followings are references that may be helpful in understanding the SAR logic and implementation. 

1. Badrul Sarwar, *et al*, "Item-based collaborative filtering recommendation algorithms", WWW, 2001.
2. Scipy (sparse matrix), url: https://docs.scipy.org/doc/scipy/reference/sparse.html
3. Asela Gunawardana and Guy Shani, "A survey of accuracy evaluation metrics of recommendation tasks", The Journal of Machine Learning Research, vol. 10, pp 2935-2962, 2009.	